# 语言模型开放题

In [1]:
import torch, torchtext
import math
import time
import random

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据集
本次开放题将与课程内容保持一致，将周杰伦从第一张专辑《Jay》到第十张专辑《跨时代》中的歌词作为训练语言模型的数据集，来训练一个语言模型，并在模型训练好后，用这个模型来创作新的歌词。

这里简介将此数据集转换成字符级循环神经网络及其变体所需要的输入格式的方法：

### 读取数据集

首先读取这个数据集，为了打印方便，我们把换行符替换成空格，并且打印前40个字符。

In [3]:
with open('Datasets/jaychou_lyrics.txt', encoding="utf-8") as f:
        corpus_chars = f.read()
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')

corpus_chars[:40]

'想要有直升机 想要和你飞到宇宙去 想要和你融化在一起 融化在宇宙里 我每天每天每'

### 建立字符索引

我们将每个字符映射成一个从0开始的连续整数作为索引，以方便之后的数据处理。

为了得到索引，我们将数据集里所有不同字符取出来，然后将其逐一映射到索引来构造词典。

接着，打印`vocab_size`，即词典中不同字符的个数，又称词典大小。

之后，将训练数据集中每个字符转化为索引，并打印前20个字符及其对应的索引。

建立字符索引之后我们就可以对于数据集进行操作了。

In [4]:
idx_to_char = list(set(corpus_chars))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
vocab_size = len(char_to_idx)
print('vocab_size:', vocab_size)
corpus_indices = [char_to_idx[char] for char in corpus_chars]
sample = corpus_indices[:20]
print('chars:', ''.join([idx_to_char[idx] for idx in sample]))
print('indices:', sample)

vocab_size: 2582
chars: 想要有直升机 想要和你飞到宇宙去 想要和
indices: [1482, 1756, 2202, 2491, 356, 630, 1901, 1482, 1756, 456, 2113, 2429, 2431, 753, 1123, 1569, 1901, 1482, 1756, 456]


# 评价指标

在本次大作业中，我们使用困惑度（perplexity）来评价语言模型的好坏，困惑度是对交叉熵损失函数做指数运算后得到的值。

* 最佳情况下，模型总是把标签类别的概率预测为1，此时困惑度为1；
* 最坏情况下，模型总是把标签类别的概率预测为0，此时困惑度为正无穷；
* 基线情况下，模型总是预测所有类别的概率都相同，此时困惑度为类别个数。

显然，任何一个有效模型的困惑度必须小于类别个数。在语言模型中，困惑度必须小于词典大小`vocab_size`。

In [5]:
import math
def perplexity(l_sum, n):
    return math.exp(l_sum / n)

# 语言模型选取

以下进入本次大作业的正题——语言模型的分析与比较。

学员需要根据每个知识点介绍之后的问题进行理论分析、模型实现、结果汇总以及指标分析，尽可能完善地阐述实验收获以回答问题，并且以撰写报告的形式汇总自己的研究成果。

## 文本预处理函数

In [6]:
# utils
def one_hot(x, n_class, dtype=torch.float32):
    """
    input shape :  (n,)
    output shape:  (n, n+class)
    """
    result = torch.zeros(x.shape[0], n_class, dtype=dtype, device=x.device)
    result.scatter_(1, x.long().view(-1, 1), 1)
    return result

def to_onehot(X, n_class):
    """
    input shape :  (batch_size, num_steps)
    output shape:  num_steps x (batch_size, n_class)
    """
    return [one_hot(X[:, i], n_class) for i in range(X.shape[1])]

def data_iter_consecutive(corpus_indices, batch_size, num_steps, device=None):
    """
    X shape:   (batch_size, num_steps)
    Y shape:   (batch_size, num_steps)
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_len = len(corpus_indices) // batch_size * batch_size
    corpus_indices = corpus_indices[: corpus_len]
    indices = torch.tensor(corpus_indices, device=device)
    indices = indices.view(batch_size, -1)  # resize成(batch_size, )
    batch_num = (indices.shape[1] - 1) // num_steps
    for i in range(batch_num):
        i = i * num_steps
        X = indices[:, i: i + num_steps]
        Y = indices[:, i + 1: i + num_steps + 1]
        yield X, Y
        
def grad_clipping(params, theta, device=None):
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    norm = torch.tensor([0.0], device=device)
    for param in params:
        norm += (param.grad.data ** 2).sum()
    norm = norm.sqrt().item()
    if norm > theta:
        for param in params:
            param.grad.data *= (theta / norm)

## RNN网络

In [7]:
# RNN
class RNNModel(torch.nn.Module):
    """
    RNN input shape   : (num_steps, batch_size, vocab_size) if batch_first=False
    RNN output shape  : (num_steps, batch_size, vocab_size) if batch_first=False
    RNN hiddens shape : (num_steps, batch_size, hidden_size) if batch_first=False
    state shape       : (num_layers * num_directions, batch_size, vocab_size)
    dense output shape: (num_steps * batch_size, vocab_size)
    """
    def __init__(self, rnn_layer):
        super(RNNModel, self).__init__()
        self.rnn = rnn_layer
        self.vocab_size = rnn_layer.input_size
        self.hidden_size = rnn_layer.hidden_size * (2 if rnn_layer.bidirectional else 1)
        self.dense = torch.nn.Linear(self.hidden_size, vocab_size)
        self.state = None

    def forward(self, inputs, state):
        """
        inputs shape: (batch_size, num_steps)
        X shape:      num_steps x (batch_size, vocab_size)
        hiddens:
        """
        X = to_onehot(inputs, self.vocab_size)
        X = torch.stack(X)
        hiddens, state = self.rnn(X, state)
        hiddens = hiddens.view(-1, hiddens.shape[-1])    # (num_steps*batch_size, hidden_size)
        output = self.dense(hiddens)
        return output, state

## 预测函数

In [8]:
def predict_rnn(prefix, num_chars, model, idx_to_char, char_to_idx, device=None):
    """
    batch_size = 1
    num_steps  = 1
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    state = None
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = torch.tensor([output[-1]], device=device).view(1,1)
        if state is not None:
            if isinstance(state, tuple):  # LSTM, state:(h, c)
                state = (state[0].to(device), state[1].to(device))
            else:
                state = state.to(device)
        (Y, state) = model(X, state)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(Y.argmax(dim=1).item())
    return ''.join([idx_to_char[i] for i in output])


## 训练函数

In [9]:
def train_and_predict_rnn(model, corpus_indices, idx_to_char, char_to_idx,
                          num_steps, batch_size,
                          num_epochs, lr, clipping_theta,
                          pred_period, pred_len, prefixes, device=None):
    loss = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.to(device)
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_consecutive(corpus_indices, batch_size, num_steps, device)  # 相邻采样
        state = None
        for X, Y in data_iter:
            if state is not None:
                # 相邻采样使用detach函数从计算图分离隐藏状态
                if isinstance(state, tuple):
                    state[0].detach_()
                    state[1].detach_()
                else:
                    state.detach_()
            (output, state) = model(X, state)  # output.shape: (num_steps * batch_size, vocab_size)
            y = torch.flatten(Y.T)             # Y.shape:      (batch_size, num_steps)
            l = loss(output, y.long())         # y.shape:      (num_steps * batch_size, 1)

            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(), clipping_theta, device)
            optimizer.step()
            l_sum += l.item() * y.shape[0]
            n += y.shape[0]

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(prefix, pred_len, model, idx_to_char, char_to_idx, device))

## 循环神经网络

在$n$元语法中，时间步$t$的词$w_t$基于前面所有词的条件概率只考虑了最近时间步的$n-1$个词。如果要考虑比$t-(n-1)$更早时间步的词对$w_t$的可能影响，模型参数的数量将随之呈指数级增长。

循环神经网络（recurrent neural network，RNN）并非刚性地记忆所有固定长度的序列，而是通过隐藏状态来存储之前时间步的信息。具体来说，在循环神经网络中，时间步$t$的隐藏变量的计算由当前时间步的输入和上一时间步的隐藏变量共同决定：

$${H}_t = \phi({X}_t {W}_{xh} + {H}_{t-1} {W}_{hh}  + {b}_h).$$

如下图所示，隐藏变量能够捕捉截至当前时间步的序列的历史信息，就像是神经网络当前时间步的状态或记忆一样。

![Image Name](https://zh.gluon.ai/_images/rnn.svg)

我们使用Pytorch中的`nn.RNN`来构造循环神经网络。在本节中，我们主要关注`nn.RNN`的以下几个构造函数参数：

* `input_size` - The number of expected features in the input x
* `hidden_size` – The number of features in the hidden state h
* `nonlinearity` – The non-linearity to use. Can be either 'tanh' or 'relu'. Default: 'tanh'
* `batch_first` – If True, then the input and output tensors are provided as (batch_size, num_steps, input_size). Default: False

这里的`batch_first`决定了输入的形状，我们使用默认的参数`False`，对应的输入形状是 (num_steps, batch_size, input_size)。

`forward`函数的参数为：

* `input` of shape (num_steps, batch_size, input_size): tensor containing the features of the input sequence. 
* `h_0` of shape (num_layers * num_directions, batch_size, hidden_size): tensor containing the initial hidden state for each element in the batch. Defaults to zero if not provided. If the RNN is bidirectional, num_directions should be 2, else it should be 1.
* `forward`函数的返回值是：
    * `output` of shape (num_steps, batch_size, num_directions * hidden_size): tensor containing the output features (h_t) from the last layer of the RNN, for each t.
    * `h_n` of shape (num_layers * num_directions, batch_size, hidden_size): tensor containing the hidden state for t = num_steps.


### 问题一：
- 解释为什么循环神经网络可以表达某时间步的词基于文本序列中所有过去的词的条件概率？
- 调节循环神经网络的超参数以及深度，分析不同超参和深度下，训练时间、语言模型困惑度（perplexity）以及创作歌词的结果等相关指标的变化，并以表格的形式进行总结。

#### RNN可表达过去词的条件概率
循环神经网络引入了隐藏变量$H_t$，`RNN`神经网络的输入包括代表历史信息的$H_{t-1}$和代表本时刻的$X_t$，$H_t$又可以作为下一时刻的输入，因此$H_t$能够捕捉截至当前时间步序列的历史信息。

### RNN模型的超参数
- bidirectional
- hidden_size
- num_layers

In [10]:
num_steps, batch_size = 35, 32
num_epochs, lr, clipping_theta = 250, 1e-3, 1e-2
pred_period, pred_len, prefixes = 50, 100, ['分开', '不分开']

#### bidirectional
本体不适合bidirectional

In [11]:
num_hiddens = 256
rnn_layer = torch.nn.RNN(input_size=vocab_size, hidden_size=num_hiddens, num_layers=1, bidirectional=True)
model = RNNModel(rnn_layer).to(device)
train_and_predict_rnn(model, corpus_indices, idx_to_char, char_to_idx,
                          num_steps, batch_size,
                          num_epochs, lr, clipping_theta,
                          pred_period, pred_len, prefixes, device)

epoch 50, perplexity 1.004757, time 1.43 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
epoch 100, perplexity 1.000438, time 1.43 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
epoch 150, perplexity 1.000089, time 1.39 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
epoch 200, perplexity 1.019067, time 1.51 sec
 - 分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开
 - 不分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开分开

#### hidden_size

In [12]:
hidden_size_seq = [64, 128, 256, 512]
for num_hiddens in hidden_size_seq:
    print("="*20, "num_hiddens = ", num_hiddens, "="*20)
    rnn_layer = torch.nn.RNN(input_size=vocab_size, hidden_size=num_hiddens, num_layers=1, bidirectional=False)
    model = RNNModel(rnn_layer).to(device)
    train_and_predict_rnn(model, corpus_indices, idx_to_char, char_to_idx,
                          num_steps, batch_size,
                          num_epochs, lr, clipping_theta,
                          pred_period, pred_len, prefixes, device)

==================== num_hiddens =  64 ====================
epoch 50, perplexity 17.574493, time 0.78 sec
 - 分开始可以 我用麻烦了 不用麻烦了 我们一直在我的时间 我们的世界 竟然先对我示了 不该开始在飘移青春  我的眼泪 不要再想一口 他们的很美 你说不是我们的招式  我们在等待 让我在等待英雄 我 不是你的
 - 不分开 我不是我的手 你不该 我不能再想你 我 不用麻烦了 我们一直人在一点 有人是一种味道叫做家 无法伪造　我们都有了空 我用力气　我们都会怕你 我不能够语气 她的睫毛弯的嘴角 让我们的感觉 一直走到  
epoch 100, perplexity 5.250626, time 0.85 sec
 - 分开苏美丽的心愿望人生活  你穿上 生死  无法 我在街上的风月 是否院子有空 在我无法 怀里 不用麻痹了心 在我边上 我们之前 我们指放松 我目光如龙 当敌人是空 我左右开弓 我气势如虹 将炮马尽用 兵
 - 不分开始不能 让我 想我拿出手机有了 我用眼泪想你 虽然我的我们都没有你 想想有没有结果  不要吵  难预测 我 何著你 别人爱的不 我不需要被崇拜 我们中了承诺 缘分开始如此应堪  石板上 低头可以 这个
epoch 150, perplexity 3.294741, time 0.78 sec
 - 分开苏美丽的秘密 玻璃的出车 无都会穿你马的走 载 那风吹在淡路 古果我自己的裁判 不该犯法移不离开始没有名和装 如果只穿上窗外的老外套　 只让我们数着紧 就想你的手机 我不是孤獨 小丑 你的影 我用古堡
 - 不分开大人相想只能承受回头 所有你可以不用麻烦了 不用麻烦了 不用麻烦了 不用麻烦了 副歌不长大家米的地上 象山青铜入侵略 我睡法绝影步两步三步四 在等待  在狂风吹 后悔在你人会的画  我用第一人称时间染
epoch 200, perplexity 2.536493, time 0.82 sec
 - 分开苏美将被摧毁 有夜点伤 把手被精准出来 这么过  我不了我 爱你不懂 你甘睡不 这个人生活不 这着了 超可了 我好了 让法吸明白在那些完全世落地对白色的帝外 将我还以开 分数必须 再来 强 如果你们有
 - 不分开大家能看说

#### num_layers

In [ ]:
num_layers_seq = [1, 2, 3]
for num_layer in num_layers_seq:
    print("=" * 20, "num_layer = ", num_layer, "=" * 20)
    rnn_layer = torch.nn.RNN(input_size=vocab_size, hidden_size=256, num_layers=num_layer, bidirectional=False)
    model = RNNModel(rnn_layer).to(device)
    train_and_predict_rnn(model, corpus_indices, idx_to_char, char_to_idx,
                          num_steps, batch_size,
                          num_epochs, lr, clipping_theta,
                          pred_period, pred_len, prefixes, device)

==================== num_layer =  1 ====================
epoch 50, perplexity 1.173704, time 1.39 sec
 - 分开 不需要勇气 我也不够不 听不到我才能够想要没有错过的方式 你在我的世界 爱从中穿越 梦与希望在飞 我向前去追 有目标就无悔 等着我超越 梦想挟带眼泪 咸咸的汉水 你我同个世界 爱从中穿越 梦与希望在
 - 不分开 寒风中尘埃 你 今 是外婆家是 不对我 你在爱我 我知道 你没有了 说 有些事关  让我有几个机  你要的很快乐开 祝想你离开 装不能不能够分辨 边 你用水的一出天气 第一个纸伞 大自己错人在吧 这
epoch 100, perplexity 1.065370, time 1.28 sec
 - 分开 不让我受伤 想快快长大 才能保护她 长大后我开始明白 为什么我 跑得比别人快 飞得比别人高 将来大家看的都是我画的漫画 大家唱的都是 我写的歌 妈妈的辛苦 不让你看见 温暖的食谱在她心里面 有空就多
 - 不分开 爱让我猜不得我 本真的狠难过 方向你看不画的手 那恐惧刻的孩子们 一起手牵手 温度的微笑 天涯灰宫 已着我心碎 清晰 原来最美丽的地下 我轻轻真的的模样 微如绝 分手在苦痛 就来难过 在伤透 心能看
epoch 150, perplexity 1.036923, time 1.30 sec
 - 分开 不让我颠倒 活着 话不不对 我不能为你的我 你不懂 说不出  海分手说不出来 蔚蓝的珊瑚海 错过瞬间苍白 当初彼此 你我都  不够成熟坦白  不应该  热情不再  你的 笑容勉强不来 爱深埋珊瑚海 
 - 不分开 爱让我猜不是我不该 不该在这时候说了我爱你 要怎么证明我没有说谎力气 请告诉我暂停算不算放弃 我只有一天的回忆 能不能给我一首歌的时间 紧紧的把那拥抱变成永远 在我的怀里你不用害怕失眠 哦 如果你想
epoch 200, perplexity 1.023478, time 1.31 sec
 - 分开 爱让我还说你 谁都了我不能 能够翻阅 的到这么我的样 我妈妈 我说爱 你要离开我知道很简单　 你说依赖是我们的阻碍 就算放开但能不能别没收我的爱　 当作我最后才明白 青 花 瓷 素胚勾勒出青花 笔锋
 - 不分开 爱所有些爱情 在

## 门控循环神经网络


门控循环神经网络（gated recurrent neural network）通过可以学习的门来控制信息的流动，可以更好地捕捉时间序列中时间步距离较大的依赖关系。


### 门控循环单元（GRU）





门控循环单元（gated recurrent unit，GRU）是一种常用的门控循环神经网络[1, 2]，它引入了重置门（reset gate）和更新门（update gate）的概念，并且计算候选隐藏状态，从而修改了循环神经网络中隐藏状态的计算方式。

* 重置门控制了上一时间步的隐藏状态如何流入当前时间步的候选隐藏状态，有助于捕捉时间序列里短期的依赖关系；
* 更新门可以控制隐藏状态应该如何被包含当前时间步信息的候选隐藏状态所更新，有助于捕捉时间序列里长期的依赖关系。

如下图所示，假设更新门在时间步$t'$到$t$（$t' < t$）之间一直近似1，那么，在时间步$t'$到$t$之间的输入信息几乎没有流入时间步$t$的隐藏状态$\boldsymbol{H}_t$。实际上，这可以看作是较早时刻的隐藏状态$\boldsymbol{H}_{t'-1}$一直通过时间保存并传递至当前时间步$t$，时间步$t$的候选隐藏状态$\tilde{\boldsymbol{H}}_t \in \mathbb{R}^{n \times h}$来辅助稍后的隐藏状态计算，使得模型更好地捕捉时间序列中时间步距离较大的依赖关系。


![Image Name](https://zh.gluon.ai/_images/gru_3.svg)


In [ ]:
hidden_size_seq = [64, 128, 256, 512]
for num_hiddens in hidden_size_seq:
    print("="*20, "num_hiddens = ", num_hiddens, "="*20)
    rnn_layer = torch.nn.GRU(input_size=vocab_size, hidden_size=num_hiddens, num_layers=1, bidirectional=False)
    model = RNNModel(rnn_layer).to(device)
    train_and_predict_rnn(model, corpus_indices, idx_to_char, char_to_idx,
                          num_steps, batch_size,
                          num_epochs, lr, clipping_theta,
                          pred_period, pred_len, prefixes, device)

### 长短期记忆（LSTM）


长短期记忆（long short-term memory，LSTM）[3] 网络，比门控循环单元的结构稍微复杂一点，它在循环神经网络的基础上引入了3个门，即输入门（input gate）、遗忘门（forget gate）和输出门（output gate），以及与隐藏状态形状相同的记忆细胞记录额外的信息。

* 遗忘门控制上一时间步的记忆细胞$\boldsymbol{C}_{t-1}$中的信息是否传递到当前时间步
* 输入门则控制当前时间步的输入$\boldsymbol{X}_t$通过候选记忆细胞$\tilde{\boldsymbol{C}}_t$如何流入当前时间步的记忆细胞


如下图所示，LSTM可以通过元素值域在$[0, 1]$的输入门、遗忘门和输出门来控制隐藏状态中信息的流动。当前时间步记忆细胞$\boldsymbol{C}_t \in \mathbb{R}^{n \times h}$的计算组合了上一时间步记忆细胞和当前时间步候选记忆细胞的信息。有了记忆细胞以后，接下来LSTM可以通过输出门来控制从记忆细胞到隐藏状态$\boldsymbol{H}_t \in \mathbb{R}^{n \times h}$的信息的流动。


![Image Name](https://zh.gluon.ai/_images/lstm_3.svg)

### 问题二：

- 请比较循环神经网络与门控循环网络之间结构的联系以及区别，并且从理论的角度描述门控循环神经网络的出现是在尝试解决循环神经网络中存在的哪几方面不足的。
- 调节门控循环单元（GRU）以及长短期记忆（LSTM）的超参数以及深度，分析不同超参和深度下，训练时间、语言模型困惑度（perplexity）以及创作歌词的结果等相关指标的变化，并以表格的形式进行总结。

## 简单循环单元（SRU）


在保持整个模型的设计思路不发生改变的情况下，深度学习的瓶颈往往在于计算。考虑到上述循环神经网络无法进行并行运算，因此往往需要需要大量的训练时间。

简单循环单元（simple recurrent unit，SRU）[4]旨在提出和探索简单快速并更具解释性的循环神经网络，因此它在门控循环网络的结构上进行了改进。


如下图所示，SRU的模型结构去除了遗忘门（forget gate）以及重置门（reset gate）对于上一时刻隐藏状态的依赖，以便于计算机实现并行化处理。

![Image Name](https://i.imgur.com/ahILNr0.png)



### 问题三：

- 通过阅读论文，试着详细分析简单循环单元（SRU）还在哪些方面进行了计算优化，尝试解决循环神经网络无法并行训练的问题，从而大大提高了训练速度。
- 调节简单循环单元（SRU）的超参数以及深度，分析不同超参和深度下，训练时间、语言模型困惑度（perplexity）以及创作歌词的结果等相关指标的变化，并以表格的形式进行总结，并与之前的网络结构进行对比分析。

### 参考文献

[1] Cho, K., Van Merriënboer, B., Bahdanau, D., & Bengio, Y. (2014). On the properties of neural machine translation: Encoder-decoder approaches. arXiv preprint arXiv:1409.1259.

[2] Chung, J., Gulcehre, C., Cho, K., & Bengio, Y. (2014). Empirical evaluation of gated recurrent neural networks on sequence modeling. arXiv preprint arXiv:1412.3555.

[3] Hochreiter, S., & Schmidhuber, J. (1997). Long short-term memory. Neural computation, 9(8), 1735-1780.

[4] Lei, T., Zhang, Y., Wang, SI., Dai, H., & Artzi, Y. (2017). Simple recurrent units for highly parallelizable recurrence. arXiv preprint arXiv:1709.02755

## 项目报告
本次大作业的终审评估以项目报告作为重要依据，开放题报告的内容和排版要求请下载文件：


[termproject1.zip](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/termproject1.zip)

需要注意的是，文件中：
- `termproject.pdf`提供了项目报告的内容格式要求
- `termproject_exp.pdf`提供了项目报告的内容排版样例


推荐使用`LaTeX`软件进行报告的撰写，相关`.tex`以及`.sty`源文件一并附于文件夹中。
